In [19]:
import pandas as pd
import ast
import os
print(os.getpid())

7546


In [3]:
file_path = "/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/filtered_osm_buildings_metadata.jsonl"
df = pd.read_json(file_path, lines=True)
len(df)

650888

In [4]:
# Count the number of non-NaN items in the 'error' column
non_nan_count = df['error'].notna().sum()
print(non_nan_count)
if non_nan_count == 0:
    df.drop(columns=['error'], inplace=True)

0


In [5]:
# Convert string representation of lists to actual lists
df['tweet language distribution'] = df['tweet language distribution'].apply(ast.literal_eval)
df = df[df['tweet language distribution'].apply(lambda x: len(x) > 0)]
len(df)

639322

In [6]:
df = df[df['building tags'].apply(lambda x: len(x) > 0)]
len(df)

639322

# how many unique building tags and remove meaningless ones

In [7]:
# Function to combine tags and classes
def combine_tags_and_classes(row):
    # Combine building tags and building class into one list
    combined = row['building tags'] + [row['building class']]
    # Remove duplicates by converting to a set and back to a list
    return list(set(combined))

# Function to split tags in a list
def split_building_tags(tags):
    if isinstance(tags, list):
        # Split each string by ";" and flatten the result
        return [tag for item in tags for tag in (item.split(";") if isinstance(item, str) else [item])]
    return tags

# Function to remove underscores from all tags in a list
def remove_underscores(tags):
    if isinstance(tags, list):
        return [tag.replace("_", " ") for tag in tags]
    return tags

def remove_slash(tags):
    if isinstance(tags, list):
        return [tag.replace("/", " ") for tag in tags]
    return tags

def remove_comma(tags):
    if isinstance(tags, list):
        return [tag.replace(",", " ") for tag in tags]
    return tags

def remove_bar(tags):
    if isinstance(tags, list):
        return [tag.replace("-", " ") for tag in tags]
    return tags

df['building tags'] = df.apply(combine_tags_and_classes, axis=1)
df['building tags'] = df['building tags'].apply(split_building_tags)
df['building tags'] = df['building tags'].apply(remove_underscores)
df['building tags'] = df['building tags'].apply(remove_slash)
df['building tags'] = df['building tags'].apply(remove_comma)
df['building tags'] = df['building tags'].apply(remove_bar)
len(df)

In [8]:
# Function to process building tags
def clean_building_tags(tags):
    if isinstance(tags, list):
        # Set of tags to be removed for efficiency
        invalid_tags = {
            'yes', ' no', '35', 'TS', 'as', 'building', 'cm', 'facade',
            'https://www.keykoleji.com/images/kampusler/kadikoy/1.jpeg',
            'no', 'other', 'ramp', 'retial', 'roof', 'room', 'route',
            'sernice', 'shine', 'structure', 'bridge', 'air shaft',
            'suspe', 'undefined','utility', 'uğur', 'vacant',
            'wall', 'yç','ts', 'viaduct', '松尾松濤ビル', 'block'
        }

        # Process tags: split by ";" and flatten
        cleaned_tags = []
        for item in tags:
            if isinstance(item, str):
                # Split by ";" and clean each tag
                sub_tags = item.split(";")
                for tag in sub_tags:
                    tag = tag.strip().lower()  # Normalize the tag
                    if tag not in invalid_tags:  # Exclude invalid tags
                        cleaned_tags.append(tag)
            else:
                cleaned_tags.append(item)
        return cleaned_tags
    return tags  # Return unmodified if not a list

# Apply the function to the DataFrame
df['building tags'] = df['building tags'].apply(clean_building_tags)
len(df)

639322

In [9]:
df = df[df['building tags'].apply(lambda x: len(x) > 0)]
len(df)

615134

In [10]:
# Filter out rows where 'building class' is 'other'
df_commercial_residential= df[df['building class'] != 'other']
df_commercial= df_commercial_residential[df_commercial_residential['building class'] == 'commercial']
df_residential= df_commercial_residential[df_commercial_residential['building class'] == 'residential']


In [11]:
# Remove residential related tags and non_commercial_tags from comercial tags
def clean_building_tags(tags):
    if isinstance(tags, list):
        # Set of tags to be removed for efficiency
        invalid_tags = {
            'apartments', 'apartment', 'apartaments', 'bungalow', 'bunker', 'cabin', 'detached',
            'dormitory', 'terrace', 'semidetached house', 'hut', 'ger', 'houseboat', 'static caravan',
            'house', 'gatehouse', 'residential', 'dwelling house', 'residences', 'residence',
            'residental', 'manor', 'villa', 'flats', 'student residence', 'townhouse', 'allotment house'
        }
        
        non_commercial_tags = {
            'allotment house', 'ambulance station', 'apartment building', 'barracks', 'block', 'boat',
            'boathouse', 'bus garage', 'bus station', 'canopy', 'carport', 'cathedral', 'cellar',
            'chapel', 'chimney', 'church', 'civic', 'college', 'construction', 'container', 'court',
            'demolished', 'disused:cafe', 'electricity', 'electronics', 'elevator', 'embassy', 'entrance',
            'fire station', 'florist', 'forsthaus', 'foto', 'fuel', 'fundacion', 'furniture', 'gangway',
            'garage', 'garages', 'gate', 'gazebo', 'gedenkstätte deutscher widerstand', 'government',
            'government office', 'guardhouse', 'hall of residence', 'hangar', 'hardware', 'hospital',
            'https://www.keykoleji.com/images/kampusler/kadikoy/1.jpeg', 'kantin vokasi',
            'kenbo commercial building', 'kindergarten', 'military', 'mixed', 'mixed use', 'monastery',
            'mosque', 'multiple', 'orangery', 'outbuilding', 'overpass', 'park', 'parking', 'pavilion',
            'place of worship', 'police', 'police station', 'porch', 'prefabricated', 'preschool',
            'produce', 'public', 'public building', 'quonset hut', 'railway', 'recreation', 'religious',
            'ruins', 'school', 'shed', 'shelter', 'ship', 'shrine', 'silo', 'stable', 'staircase', 'stairs',
            'station', 'storage tank', 'store', 'studio', 'studios', 'subdistrict office', 'subdivided house',
            'substation', 'subway entrance', 'synagogue', 'tank', 'tech cab', 'temple', 'tent',
            'transformer tower', 'university', 'ventilation shaft', 'voiture publicitaire', 'water tower',
            'https:  www.keykoleji.com images kampusler kadikoy 1.jpeg',
        }


        # Process tags: split by ";" and flatten
        cleaned_tags = []
        for item in tags:
            if isinstance(item, str):
                # Split by ";" and clean each tag
                sub_tags = item.split(";")
                for tag in sub_tags:
                    tag = tag.strip().lower()  # Normalize the tag
                    if tag not in invalid_tags and tag not in non_commercial_tags:  # Exclude invalid and non-commercial tags
                        cleaned_tags.append(tag)
            else:
                cleaned_tags.append(item)
        return cleaned_tags
    return tags  # Return unmodified if not a list

# Only keep residential related tags
def keep_building_tags(tags):
    # Set of valid tags for efficiency
    valid_tags = {
        'apartments', 'apartment', 'apartaments', 'bungalow', 'bunker', 'cabin', 'detached',
        'dormitory', 'terrace', 'semidetached house', 'hut', 'ger', 'houseboat', 'static caravan',
        'house', 'gatehouse', 'residential', 'dwelling house', 'residences', 'residence',
        'residental', 'manor', 'villa', 'flats', 'student residence', 'townhouse', 'allotment house',
    }

    if isinstance(tags, list):
        # Flatten, normalize, and filter tags
        cleaned_tags = [
            tag.strip().lower() for item in tags if isinstance(item, str)
            for tag in item.split(";") if tag.strip().lower() in valid_tags
        ]
        return cleaned_tags
    return tags  # Return unmodified if not a list



df_commercial['building tags'] = df_commercial['building tags'].apply(clean_building_tags)
df_residential['building tags'] = df_residential['building tags'].apply(keep_building_tags)

df = pd.concat([df_commercial, df_residential])
len(df)

/tmp/ipykernel_7546/3807581514.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_commercial['building tags'] = df_commercial['building tags'].apply(clean_building_tags)
/tmp/ipykernel_7546/3807581514.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_residential['building tags'] = df_residential['building tags'].apply(keep_building_tags)


554553

In [13]:
def preprocess_building_names(names):
    if isinstance(names, list):
        # Normalize case and strip whitespace
        names = [name.title().strip() for name in names]
        # Split compound names
        names = [subname for name in names for subname in name.split(";")]
        # Deduplicate
        names = list(set(names))
        return names
    return names


# df['building names'] = df.apply(combine_and_deduplicate, axis=1)
df['building names'] = df['building names'].apply(preprocess_building_names)
len(df)

554553

In [14]:
df = df[df['building names'].apply(lambda x: len(x) > 0)]
len(df)

94928

In [16]:
df.drop(columns=['index'], inplace=True)
# df.drop(columns=['building id'], inplace=True)
df.drop(columns=['building operators'], inplace=True)

In [ ]:
# df['building city'].unique()

# cities_language_unacceptable = ['Moscow', 'Cairo', 'Istanbul', 'Amsterdam', 'Kyoto', 'Tokyo', 'HongKong', 'Jakarta', 'GuangZhou', 'Shanghai', 'Nanjing', 'Islamabad', 'Nairobi', 'Beijing', 'Wuhan', 'Tehran', 'Changsha', 'Shenzhen', 'Qingdao']

In [17]:
# Save DataFrame to JSONL
output_path = '/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/useful_buildings.jsonl'
df.to_json(output_path, orient='records', lines=True, force_ascii=False)

In [ ]:
df['building class'].value_counts()

In [ ]:
df

In [ ]:
temp = df[df['building names'].apply(lambda x: len(x) == 1)]

temp = temp[temp['building tags'].apply(lambda x: len(x) == 2)]

len(temp)

In [ ]:
temp['building class'].value_counts()

In [ ]:
temp = temp[temp['tweet language distribution'].apply(lambda x: len(x) < 11)]
len(temp)

In [ ]:
temp['building class'].value_counts()

In [ ]:
temp['building city'].nunique()